In [7]:
import os
import math
import string
import random
import math
import collections
from collections import defaultdict
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

stopWords =['subject:','from:', 'date:', 'newsgroups:', 'message-id:', 'lines:', 'path:', 'organization:', 
            'would', 'writes:', 'references:', 'article', 'sender:', 'nntp-posting-host:', 'people', 
            'university', 'think', 'xref:', 'cantaloupe.srv.cs.cmu.edu', 'could', 'distribution:', 'first', 
            'anyone','world', 'really', 'since', 'right', 'believe', 'still', 
            "max>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'"]

data_dir = "/Users/vaishnavibalay/Desktop/20_newsgroups"
classes= os.listdir(data_dir)
data = []
for category in classes:
    category_path = os.path.join(data_dir, category)
    files = os.listdir(category_path)
    for file in files:
        with open(os.path.join(category_path, file), "r", encoding="utf-8", errors="ignore") as f:
            data.append((f.read(), classes.index(category)))

# Preprocess the data
def preprocess_data(data):
    stemmer = PorterStemmer()
    preprocessed_data = []
    for content, label in data:
        content = content.lower()
        content = content.translate(str.maketrans('', '', string.punctuation))
        words = content.split()
        words = [stemmer.stem(word) for word in words if word not in stopWords]  # Apply stemming
        preprocessed_data.append((words, label))
    return preprocessed_data


preprocessed_data = preprocess_data(data)
print(len(preprocessed_data))

# Split the data into training and testing sets
def split_data(data, split_ratio=0.5):
    random.shuffle(data)
    train_data = []
    test_data = []
    for c in range(len(classes)):  # Iterate over class indices
        c_data = [(d, l) for d, l in data if l == c]  # Filter data by class index
        random.shuffle(c_data)
        train_size = int(len(c_data) * split_ratio)
        train_data += c_data[:train_size]
        test_data += c_data[train_size:]
    return train_data, test_data


train_data, test_data = split_data(preprocessed_data)
print(len(test_data))





19997
9999


In [8]:
# Train the Naive Bayes classifier
def train_naive_bayes(train_data):
    # Count the number of documents in each class
    class_counts = [0] * len(classes)
    for _, label in train_data:
        class_counts[label] += 1
    total_documents = len(train_data)

    # Calculate the prior probabilities for each class
    priors = [count / total_documents for count in class_counts]

    # Count the frequency of each feature in each class
    feature_counts = [collections.defaultdict(int) for _ in range(len(classes))]
    class_totals = [0] * len(classes)
    for words, label in train_data:
        for word in words:
            feature_counts[label][word] += 1
            class_totals[label] += 1

    # Calculate the conditional probabilities for each feature given each class
    cond_probs = [{} for _ in range(len(classes))]
    vocab = set(feature_counts[0].keys()).union(set(feature_counts[1].keys()))

    for i in range(len(classes)):
        for word in vocab:
            count = feature_counts[i][word]
            total = class_totals[i]
            cond_probs[i][word] = (count + 1) / (total + len(vocab))

    return classes, cond_probs




In [9]:
def test_naive_bayes(test_data, priors, cond_probs):
    correct = 0
    classes = range(len(priors))
    for words, true_label in test_data:
        posteriors = []
        for label in classes:
            try:
                posteriors.append(math.log(float(priors[label])))
            except ValueError:
                posteriors.append(-float('inf'))
        for word in words:
            for label in classes:
                if word in cond_probs[label]:
                    posteriors[label] += math.log(cond_probs[label][word])
                else:
                    posteriors[label] -= math.log(sum(cond_probs[label].values()))
        pred_label = posteriors.index(max(posteriors))
        if pred_label == true_label:
            correct += 1
    accuracy = correct / len(test_data)
    return accuracy

classes, cond_probs = train_naive_bayes(train_data)
priors = [count / len(train_data) for count in collections.Counter(map(lambda x: x[1], train_data)).values()]

# Test the Naive Bayes classifier
accuracy = test_naive_bayes(test_data, priors, cond_probs)

print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 78.23%
